<a href="https://colab.research.google.com/github/d556645fw/Side-project/blob/main/%E6%99%BA%E6%85%A7%E5%AE%A2%E6%9C%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentence-transformers

In [ ]:
import sqlite3

conn = sqlite3.connect('faq.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS faq (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question TEXT,
    answer TEXT,
    tags TEXT
)
''')

faqs = [
    ("如何重設密碼？", "請至設定 > 安全性中選擇重設密碼。", "帳號,密碼"),
    ("客服的聯絡方式是什麼？", "您可以透過 email@example.com 聯絡客服。", "聯絡,客服"),
    ("如何取消訂閱？", "請至帳號設定頁面中選擇取消訂閱。", "訂閱,取消"),
    ("有沒有提供退款？", "我們提供7天內退款保障，請聯絡客服。", "退款,保證")
]

cursor.executemany("INSERT INTO faq (question, answer, tags) VALUES (?, ?, ?)", faqs)
conn.commit()

In [ ]:
from sentence_transformers import SentenceTransformer, util

# 使用中文效果不錯的模型
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def get_faqs():
    cursor.execute("SELECT question, answer FROM faq")
    return cursor.fetchall()

def find_best_match(user_question, faqs, top_k=1):
    questions = [q for q, _ in faqs]
    embeddings = model.encode(questions + [user_question], convert_to_tensor=True)

    # 最後一個是 user_question
    similarity = util.cos_sim(embeddings[-1], embeddings[:-1])
    best_idx = similarity.argmax()
    score = similarity[0][best_idx].item()

    # 設定最低相似度門檻
    if score < 0.4:
        return "找不到相關答案，請聯絡客服進一步查詢。"
    return faqs[best_idx][1]

In [ ]:
while True:
    user_input = input("請輸入問題（輸入 'exit' 離開）：")
    if user_input.lower() == 'exit':
        break
    faqs = get_faqs()
    answer = find_best_match(user_input, faqs)
    print("AI 客服回答：", answer)